<a href="https://colab.research.google.com/github/KatBotkowska/CodersCrew/blob/master/Netflix_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pobranie danych z Kaggle


###https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166

##Importy - do przeniesienia z treści niżej

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#podłączenie google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
root_path = 'gdrive/My Drive/netflix/'

In [ ]:
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tightprince","key":"88e79141903d7fcb72c9bf7b07695fcf"}'}

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


##Pobranie i rozpakowanie danych kaggle - netflix

In [ ]:
!kaggle datasets download -d shivamb/netflix-shows

netflix-shows.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -q netflix-shows.zip -d [netflix]

replace [netflix]/netflix_titles.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


##Pobranie i rozpakowanie danych kaggle - imdb

In [ ]:
!kaggle datasets download -d stefanoleone992/imdb-extensive-dataset

 91% 75.0M/82.3M [00:00<00:00, 73.1MB/s]
100% 82.3M/82.3M [00:00<00:00, 94.3MB/s]


In [ ]:
!unzip -q imdb-extensive-dataset.zip -d [netflix]

##Pobranie i rozpakowanie danych kaggle - movielens

In [ ]:
!kaggle datasets download -d grouplens/movielens-20m-dataset

movielens-20m-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -q movielens-20m-dataset.zip -d [netflix]

replace [netflix]/genome_scores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


##Załadowanie dataframes

In [ ]:
import pandas as pd
#netflix_titles = pd.read_csv('/content/[netflix]/netflix_titles.csv')

In [ ]:
imdb_movies = pd.read_csv('/content/[netflix]/IMDb movies.csv')
imdb_names = pd.read_csv('/content/[netflix]/IMDb names.csv')
imdb_ratings = pd.read_csv('/content/[netflix]/IMDb ratings.csv')
imdb_title_principals = pd.read_csv('/content/[netflix]/IMDb title_principals.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
movielens_genome_scores = pd.read_csv('/content/[netflix]/genome_scores.csv')
movielens_genome_tags = pd.read_csv('/content/[netflix]/genome_tags.csv')
movielens_link = pd.read_csv('/content/[netflix]/link.csv')
movielens_movie = pd.read_csv('/content/[netflix]/movie.csv')
movielens_rating = pd.read_csv('/content/[netflix]/rating.csv')
movielens_tag = pd.read_csv('/content/[netflix]/tag.csv')

FileNotFoundError: ignored

##Czyszczenie danych
zamiana wiersza z TV movie 2019 na 2019
dtypes z object na właściwe
NaN??
gatunki zamiana ze stringa na listy tagów


#Netflix - podstawowe dane


In [ ]:
netflix_titles.head()

NameError: ignored

In [ ]:
netflix_titles.shape
#rozmiary  - wiersze i kolumny

In [ ]:
netflix_titles.info()


In [ ]:
netflix_titles.describe()
#trochę bez sensu, nie ma tutaj dużo danych numerycznych do podsumowań

##Netflix dla 2019

In [ ]:
netflix_2019 = netflix_titles.iloc[:,:][netflix_titles.release_year==2019]
netflix_2019.head()

In [ ]:
netflix_2019.shape

In [ ]:
imdb_movies.tail()

##IMDB dla 2019

In [ ]:
imdb_2019 = imdb_movies.iloc[:,:][imdb_movies.year==2019]
imdb_2019.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
44619,tt0385887,Motherless Brooklyn - I segreti di una città,Motherless Brooklyn,2019,2019-11-07,"Crime, Drama, Mystery",144,USA,"English, French",Edward Norton,"Edward Norton, Jonathan Lethem",Warner Bros. Pictures,"Edward Norton, Gugu Mbatha-Raw, Alec Baldwin, ...","In 1950s New York, a lonely private detective ...",6.8,34759,$ 26000000,$ 9277736,$ 18477736,60.0,319.0,178.0
46723,tt0437086,Alita - Angelo della battaglia,Alita: Battle Angel,2019,2019-02-14,"Action, Adventure, Sci-Fi",122,"USA, Japan, Canada","English, Spanish",Robert Rodriguez,"James Cameron, Laeta Kalogridis",Twentieth Century Fox,"Rosa Salazar, Christoph Waltz, Jennifer Connel...","A deactivated cyborg is revived, but cannot re...",7.3,217183,$ 170000000,$ 85710210,$ 404852543,53.0,2757.0,368.0
46892,tt0441881,La battaglia di Long Tan,Danger Close: The Battle of Long Tan,2019,2019-08-08,"Action, Drama, War",118,Australia,English,Kriv Stenders,"Stuart Beattie, James Nicholas",Deeper Water,"Travis Fimmel, Toby Blome, Alexander England, ...","In August 1966, in a Vietnamese rubber plantat...",6.8,8131,AUD 24000000,NaN,$ 2078370,NaN,204.0,31.0
47132,tt0448115,Shazam!,Shazam!,2019,2019-04-03,"Action, Adventure, Comedy",132,"USA, Canada","English, Spanish",David F. Sandberg,"Henry Gayden, Henry Gayden",Warner Bros.,"Zachary Levi, Mark Strong, Asher Angel, Jack D...",A newly fostered young boy in search of his mo...,7.1,240858,$ 100000000,$ 140371656,$ 365971656,71.0,2009.0,379.0
49759,tt0783640,Era mio figlio,The Last Full Measure,2019,2020-04-02,"Drama, War",116,USA,English,Todd Robinson,Todd Robinson,Foresight Unlimited,"Sebastian Stan, Alison Sudol, Asher Miles Fall...","Thirty-four years after his death, Airman Will...",6.7,5449,$ 20000000,$ 2949212,$ 3364426,51.0,163.0,44.0


##Łączenie IMDB i Netflix dla 2019

In [ ]:
netflix_imdb_2019 = pd.merge(netflix_2019, imdb_2019, on=['title'])
netflix_imdb_2019.info()

In [ ]:
netflix_imdb_2019.shape

In [ ]:
netflix_imdb_2019

##Łączenie IMDB i Netflix dla wszystkich danych

In [ ]:
netflix_imdb_all = pd.merge(netflix_titles, imdb_movies, on=['title'])
netflix_imdb_all.info()

In [ ]:
netflix_imdb_all.head()

In [ ]:
imdb_movies.columns

# Szeregi czasowe - jak przez lata zmieniała się liczba wydawanych filmów? - Ewa

In [ ]:
import pandas as pd
imdb_movies = pd.read_csv('/content/[netflix]/IMDb movies.csv')

imdb_movies.head()

In [ ]:
imdb_movies.dtypes

In [ ]:
imdb_movies.groupby('year').imdb_title_id.count()

In [ ]:
imdb_movies[imdb_movies.year == 'TV Movie 2019']

In [ ]:
imdb_movies.loc[83917, 'year'] = 2019
imdb_movies.year = imdb_movies.year.astype('int')
imdb_movies.dtypes

In [ ]:
data_year_plot = imdb_movies.groupby('year').imdb_title_id.count()
data_year_plot.tail(20)

In [ ]:
import matplotlib.pyplot as plt

data_year_plot.plot()
plt.show()

In [ ]:
data_year_plot_from2004 = imdb_movies[imdb_movies.year > 2004].groupby('year').imdb_title_id.count()
data_year_plot_from2004.plot()

# chyba nie ma sensu brać do analizy roku 2020. zastanawiam się nad 2019 - czy przez covida tak mało, czy też braki danych.

In [ ]:
data_year_plot_to2009 = imdb_movies[imdb_movies.year <= 2009].groupby('year').imdb_title_id.count()
data_year_plot_to2009.plot()

In [ ]:
imdb_movies.loc[83917, 'date_published'] = '2019-01-01' #ustawiam 1 stycznia jako datę publikacji tego dziwnego filmu

pd.to_datetime(imdb_movies['date_published'])

In [ ]:
imdb_movies['date_published'].dt.year #nie wiem czemu wyskakuje błąd, dkoro wyżej widać, że ta kolumna już jest typem datetime

#Na lata jakie rodzaje filmów dominują, jakie trendy w gatunkach - KASIA

In [ ]:

imdb_movies = pd.read_csv('/content/[netflix]/IMDb movies.csv')

##Najczęściej występujące gatunki

In [ ]:
imdb_movies['genre'].value_counts().head(10)

In [ ]:
ax = imdb_movies['genre'].value_counts().head().plot.barh( rot=0)

##Ilość gatunków w poszczególnych latach
Widać, że gatunki to 'tagi' opisujące film, do jednego filmu przypisuje się coraz więcej gatunków.

In [ ]:
#ilość gatunków w poszczególnych latach
imdb_movies.groupby('year').genre.count().tail(20)

##Lista wszystkich gatunków

In [ ]:
genres = imdb_movies['genre'].unique().tolist()
genres_all=[]
for genre in genres:
    for elem in genre.split(','):
        genres_all.append(elem.strip())
genres_all=list(set(genres_all))
genres_all

##Unikalne gatunki - najbardziej popularne

In [ ]:
genres = imdb_movies['genre'].tolist()
genres_count=[]
for genre in genres:
    for elem in genre.split(','):
        genres_all.append(elem.strip())
genres_count=list(genres_all)
from collections import Counter
genres_count_all=pd.DataFrame.from_dict(Counter(genres_count), orient='index')
genres_count_all.rename(columns={'index':'genres', 0:'occurences'}, inplace=True)
genres_count_all.sort_values(by='occurences', ascending=False)


In [ ]:
ax = genres_count_all.sort_values(by='occurences', ascending=False).head().plot.barh(rot=1, title='Genres by occurences')
ax.set_xlabel('occurences')
#ax.set_ylabel()


##Gatunki na lata

In [ ]:
imdb_movies['genre_list']=imdb_movies['genre'].str.split(',', n=5)
#imdb_movies['genre_list'].value_counts().head(10)

In [ ]:
genres_by_years=imdb_movies[['genre', 'year']].groupby(['year', 'genre'], as_index=True)['genre'].count()
genres_by_years.tail(100)

In [ ]:
#dla roku 2019 zamiana str gatunków na listę
genres_split_by_years=imdb_movies[['genre', 'year', 'imdb_title_id']]
genres_split_by_years[genres_split_by_years['year']=='2019']
genres = genres_split_by_years['genre']

new_genres=[]
for genre in genres.to_list():
    genre = genre.split(',')
    new_genres.append(genre)

genres_split_by_years['new_genre']=new_genres
genres_split_by_years


In [ ]:
genres_in_2019 = genres_split_by_years[genres_split_by_years['year']=='2019']
genres_2019=genres_in_2019['new_genre'].to_list()
genres_2019

In [ ]:
genres_count=[]
genres_all=[]
for genres in genres_2019:
    for genre in genres:
        genres_all.append(genre)
genres_count_2019=list(genres_all)
from collections import Counter
genres_count_all_2019=pd.DataFrame.from_dict(Counter(genres_count_2019), orient='index')
genres_count_all_2019.rename(columns={'index':'genres', 0:'occurences'}, inplace=True)

genres_count_all_2019.sort_values(by='occurences', ascending=False)

In [ ]:
#Genres by years

table={}
years = ['2010', '2011', '2012','2013','2014','2015','2016', '2017','2018','2019']
genres = ['Drama', 'Comedy', 'Romance', 'Action', 'Thriller','Crime', 'Horror']

for year in years:

    genres_in_year = genres_split_by_years[genres_split_by_years['year']==year]['new_genre']
    genres_in_year=[item.strip() for sublist in genres_in_year for item in sublist]
    table[year]={}
    for genre in genres: ##genres_all nie działa???
        table[year][genre]=genres_in_year.count(genre)

genres_in_years = pd.DataFrame(table)
genres_in_years
   



In [ ]:
genres_in_years.T.plot()

In [ ]:
#sprawdzenie roku 2010
genres_in_2010 = genres_split_by_years[genres_split_by_years['year']=='2010']
genres_2010=genres_in_2010['new_genre'].to_list()
genres_2010

In [ ]:
data_year_plot = imdb_movies.groupby('year').imdb_title_id.count()
data_year_plot.tail(20)

# Jakie kraje dominują w produkcji filmów  - które wypuszczają najwięcej filmów, i z 5 największych zrobić szeregi czasowe - GRZESIEK

In [ ]:
import pandas as pd
imdb_movies = pd.read_csv('/content/[netflix]/IMDb movies.csv')

In [ ]:
''' 
Po pierwsze, te kraje są bardzo dziwne. Po drugie nie jestem 
w stanie określić która firma produkcyjna należy do danego kraju. 
Po trzecie te wszystkie kraje pokazują gdzie filmy zostały nakręcone.
'''
imdb_movies['country'].value_counts().head(10)

In [ ]:
'''
Te wykresy są okej, ale później jest coś nie tak...
'''
import matplotlib.pyplot as plt
import seaborn as sn
chart_country = imdb_movies['country'].value_counts().head(10)
chart_country.plot(kind='barh', figsize=(16,8), fontsize=13)
plt.title("Where the movies were filmed", fontsize=10)
plt.xlabel('Number of movies')
plt.ylabel('Countries')
plt.show()

In [ ]:
'''
Jak tutaj widzimy że daty się pomieszały. Spróbowałem sortować datę i nie jestem w stanie tego naprawić.
'''
imdb_movies.groupby('year').country.count().plot()

In [ ]:
countries_new = imdb_movies[['year', 'country']]
countries_new

In [ ]:
'''
Teraz sprawdzam produkcję filmu. Jak widać że czołowe firmy należą do USA oprócz Mosfilm i Canal+.
'''
imdb_movies['production_company'].value_counts().head(20)

In [ ]:
imdb_production = imdb_movies['production_company'].value_counts()
imdb_production.iloc[:10].plot(kind='barh',figsize=(16,8),fontsize=13)
plt.title("production companies and number of movies",fontsize=15)
plt.xlabel('number of movies',fontsize=14)
sn.set_style("whitegrid")

In [ ]:
'''
Tutaj niestety jest za mało kolumn żeby określić w których krajach produkowali najwięcej 
filmików, więc sprawdzam które produkcje filmowe zostały najbardziej ocenione.
'''
imdb_movies.corr()

In [ ]:
movie_sliced = imdb_movies[0:1000]


plt.figure(figsize=(15,15))
sn.swarmplot(x='avg_vote', y='country', data = movie_sliced)
plt.title ('Which countries have the best productions?', fontsize=20, fontweight='bold')
plt.xlabel('Score')
plt.ylabel('Country')
plt.show()

#Ile przypadków, że scenarzysta jest jednocześnie reżyserem w filmie - EWA

In [ ]:
imdb_movies = pd.read_csv('/content/[netflix]/IMDb movies.csv')

In [ ]:
imdb_movies[imdb_movies.writer == imdb_movies.director].director.count()
# to tak prosto, czyli odpowiada na postawione pytanie, 
# ale tylko wtedy jeśli jest jeden reżyser i jeden scenarzysta.

In [ ]:
imdb_movies['writer_1'] = imdb_movies.writer.str.split(pat=',', expand=True)[0]
imdb_movies['writer_2'] = imdb_movies.writer.str.split(pat=',', expand=True)[1]
imdb_movies[['writer_1', 'writer_2']]

In [ ]:
imdb_movies['director_1'] = imdb_movies.director.str.split(pat=',', expand=True)[0]
imdb_movies['director_2'] = imdb_movies.director.str.split(pat=',', expand=True)[1]
imdb_movies[['director_1', 'director_2']]

In [ ]:
# tu taki brzydki długi warunek :)
# ale wydaje mi się, że dobrze porównuje i zlicza wyżej utworzone kolumny

imdb_movies[(imdb_movies.writer_1 == imdb_movies.director_1) | 
        (imdb_movies.writer_1 == imdb_movies.director_2) | 
        (imdb_movies.writer_2 == imdb_movies.director_1) | 
        (imdb_movies.writer_2 == imdb_movies.director_2)].imdb_title_id.count()

#Reżyser - z największymi średnimi głosów, największa liczba głosów - KASIA

In [ ]:
imdb_movies = pd.read_csv('/content/[netflix]/IMDb movies.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
imdb_movies.columns

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics'],
      dtype='object')

In [ ]:
#Reżyserzy z największą średnią ocen, którzy nakręcili więcej niż 5 filmów
imdb_directors_av_votes=imdb_movies[['director','avg_vote']].groupby('director').agg(['count', 'mean']).reset_index()
imdb_directors_av_votes[imdb_directors_av_votes['avg_vote']['count']>5].sort_values(by=('avg_vote','mean'), ascending=False).reset_index().head(10)

index           director avg_vote          
                               count      mean
0   5749  Christopher Nolan       11  8.218182
1  28854       Satyajit Ray       28  8.025000
2   1753   Andrei Tarkovsky        7  8.014286
3  12193     Hayao Miyazaki       12  8.008333
4  29269       Sergio Leone        7  7.928571
5  12725      Humayun Ahmed        7  7.900000
6  13739      James Cameron        7  7.885714
7  25895  Quentin Tarantino       12  7.858333
8  31398    Tengiz Abuladze        6  7.850000
9  29214          Sergey A.       24  7.850000

In [ ]:
#reżyserzy, którzy kręcą dużo i źle ;)
imdb_directors_av_votes[imdb_directors_av_votes['avg_vote']['count']>10].sort_values(by=('avg_vote','mean'), ascending=True).head(10)

director avg_vote          
                          count      mean
4153       Brett Kelly       18  1.533333
26614       Rene Perez       18  2.805556
14121       Jared Cohn       16  2.943750
20607      Mark Atkins       12  3.066667
1831      Andrew Jones       24  3.241667
7102    David DeCoteau       44  3.259091
30577  Steven M. Smith       12  3.266667
6933    Daryush Shokof       11  3.318182
18975      Leigh Scott       12  3.341667
27025      Rick Sloane       11  3.400000

In [ ]:
#jakie oceny dostają reżyserzy którzy idą 'na ilość'
imdb_directors_av_votes[imdb_directors_av_votes['avg_vote']['count']>5].sort_values(by=('avg_vote','count'), ascending=False).head(10)

director avg_vote          
                           count      mean
15006      Jesús Franco       87  4.718391
21840    Michael Curtiz       85  6.543529
19114   Lesley Selander       78  6.214103
19360       Lloyd Bacon       73  6.232877
33646  William Beaudine       70  5.861429
26955    Richard Thorpe       68  6.263235
15682         John Ford       67  6.710448
11306    Gordon Douglas       64  6.175000
26370       Raoul Walsh       61  6.590164
21702      Mervyn LeRoy       59  6.689831

In [ ]:
#Reżyserzy których filmy otrzymują średnio najwięcej głosów, którzy nakręcili więcej niż 5 filmów
imdb_directors_sum_votes=imdb_movies[['director','votes']].groupby('director').agg(['count', 'sum']).reset_index()
imdb_directors_sum_votes['votes_per_film'] = imdb_directors_sum_votes['votes']['sum']/imdb_directors_sum_votes['votes']['count']
imdb_directors_sum_votes['votes_per_film']=imdb_directors_sum_votes['votes_per_film'].astype('int')
imdb_directors_sum_votes[imdb_directors_sum_votes['votes']['count']>5].sort_values(by=('votes_per_film'), ascending=False).head(10)

director votes           votes_per_film
                                       count       sum               
5749                 Christopher Nolan    11  11653144        1059376
25895                Quentin Tarantino    12   8458703         704891
13739                    James Cameron     7   4915763         702251
7141                     David Fincher    10   6944421         694442
25197                    Peter Jackson    13   7304418         561878
10749                     George Lucas     6   3364510         560751
2353          Anthony Russo, Joe Russo     6   3006253         501042
13336                      J.J. Abrams     6   2881528         480254
18639  Lana Wachowski, Lilly Wachowski     6   2871445         478574
34464                      Zack Snyder     8   3410835         426354

#Scenarzysta - jw - GRZESIEK

In [ ]:
imdb_movies = pd.read_csv('/content/[netflix]/IMDb movies.csv')

In [ ]:
imdb_movies.columns

In [ ]:
imdb_writers_av_votes=imdb_movies[['writer','avg_vote']].groupby('writer').agg(['count', 'mean']).reset_index()
imdb_writers_av_votes[imdb_writers_av_votes['avg_vote']['count']>10].sort_values(by=('avg_vote','mean'), ascending=False).reset_index().head(10)

In [ ]:
imdb_writers_av_votes[imdb_writers_av_votes['avg_vote']['count']>10].sort_values(by=('avg_vote','mean'), ascending=True).head(10)

In [ ]:
imdb_writers_av_votes[imdb_writers_av_votes['avg_vote']['count']>5].sort_values(by=('avg_vote','count'), ascending=False).head(10)

In [ ]:
imdb_writers_sum_votes=imdb_movies[['writer','votes']].groupby('writer').agg(['count', 'sum']).reset_index()
imdb_writers_sum_votes['votes_per_film'] = imdb_writers_sum_votes['votes']['sum']/imdb_writers_sum_votes['votes']['count']
imdb_writers_sum_votes['votes_per_film']=imdb_writers_sum_votes['votes_per_film'].astype('int')
imdb_writers_sum_votes[imdb_writers_sum_votes['votes']['count']>5].sort_values(by=('votes_per_film'), ascending=False).head(10)

#Reżyser, Scenarzysta, aktor - jakie przynoszą dochody dla filmu - dochód w USA i dochód na świecie, średni budżet ich filmów -KASIA

#Reżyser - dochód z filmów

##Reżyserzy z największą ilością filmów

In [ ]:
#Reżyserzy z największą ilością filmów
imdb_movies['director'].value_counts().head(10)

##Reżyserzy, których filmy osiągnęły największy dochód na świecie - dot. dochodów w USD - może zrobić jeszcze INR rupię indyjską? Bolywood? Może filmy kręcone w Chinach (jaka waluta?) jaki mają dochód?

In [ ]:
imdb_movies['worlwide_gross_income']

In [ ]:
#Wyczyszczenie NaN
imdb_movies['worlwide_gross_income'].replace(np.nan, '$ 0', inplace=True)

In [ ]:
#poprzez split oddzielenie waluty od wartości
imdb_movies['worlwide_gross_income']=imdb_movies['worlwide_gross_income'].apply(lambda x: x.split())


In [ ]:
#Wydzielenie walut do osobnej kolumny
imdb_movies['currency_for_wwincome']=imdb_movies['worlwide_gross_income'].apply(lambda x: x[0])

In [ ]:
#Waluty występujące w worlwide gross income
imdb_movies['currency_for_wwincome'].unique()

In [ ]:
#kolumna z worlwide gross income tylko z wartościami
imdb_movies['worlwide_gross_income']=imdb_movies['worlwide_gross_income'].apply(lambda x: x[1])

In [ ]:
#zmiana stringów na int
imdb_movies['worlwide_gross_income']=imdb_movies['worlwide_gross_income'].astype(int)

In [ ]:
#nowa df z danymi gdzie worlwide_gross_income w dolarach
imdb_movies_USD = imdb_movies[imdb_movies['currency_for_wwincome']=='$']

In [ ]:
#zmiana typu z obj na int
imdb_movies_USD['worlwide_gross_income'] =imdb_movies_USD['worlwide_gross_income'].astype(int)

In [ ]:
imdb_movies_USD['worlwide_gross_income']

In [ ]:
#Wyczyszczenie $ i spacji - tak się nie da bo różne waluty
#imdb_movies['worlwide_gross_income']=imdb_movies['worlwide_gross_income'].apply(lambda x: x[2:])


In [ ]:
#Reżyserzy z największym dochodem w dolarach
imdb_movies_USD.groupby('director').agg({'imdb_title_id':'count','worlwide_gross_income':'sum'}).reset_index().rename(columns={'imdb_title_id':'films_count'}).sort_values(by='worlwide_gross_income', ascending=False).head(20)

##Bollywood

In [ ]:
##Bolywood
#nowa df z danymi gdzie worlwide_gross_income w rupii indyjskiej 
imdb_movies_INR = imdb_movies[imdb_movies['currency_for_wwincome']=='INR']

In [ ]:
#zmiana typu z obj na int
imdb_movies_INR['worlwide_gross_income'] =imdb_movies_INR['worlwide_gross_income'].astype(int)

In [ ]:
#Reżyserzy z największym dochodem w rupii indyjskiej
imdb_movies_INR.groupby('director').agg({'imdb_title_id':'count','worlwide_gross_income':'sum'}).reset_index().rename(columns={'imdb_title_id':'films_count'}).sort_values(by='worlwide_gross_income', ascending=False).head(10)

##Reżyserzy dochód z filmów w USA

In [ ]:
imdb_movies[imdb_movies['usa_gross_income'].notnull()].head()

In [ ]:
imdb_movies['usa_gross_income']

In [ ]:
#Wyczyszczenie NaN
imdb_movies['usa_gross_income'].replace(np.nan, '$ 0', inplace=True)

In [ ]:
#poprzez split oddzielenie waluty od wartości
imdb_movies['usa_gross_income']=imdb_movies['usa_gross_income'].apply(lambda x: x.split())


In [ ]:
#Wydzielenie walut do osobnej kolumny
imdb_movies['currency_for_usaincome']=imdb_movies['usa_gross_income'].apply(lambda x: x[0])

In [ ]:
#Waluty występujące w USA gross income
imdb_movies['currency_for_usaincome'].unique()

In [ ]:
#kolumna z USA gross income tylko z wartościami
imdb_movies['usa_gross_income']=imdb_movies['usa_gross_income'].apply(lambda x: x[1])

In [ ]:
#zmiana stringów na int
imdb_movies['usa_gross_income']=imdb_movies['usa_gross_income'].astype(int)

In [ ]:
#nowa df z danymi gdzie usa_gross_income w dolarach
imdb_movies_USD = imdb_movies[imdb_movies['currency_for_usaincome']=='$']

In [ ]:
#zmiana typu z obj na int
imdb_movies_USD['usa_gross_income'] =imdb_movies_USD['usa_gross_income'].astype(int)

In [ ]:
imdb_movies_USD['usa_gross_income']

In [ ]:
#Reżyserzy z największym dochodem w dolarach
imdb_movies_USD.groupby('director').agg({'imdb_title_id':'count','usa_gross_income':'sum'}).reset_index().rename(columns={'imdb_title_id':'films_count'}).sort_values(by='usa_gross_income', ascending=False).head(20)

#Scenarzysta - dochody z filmów, na świecie, w USA

##Scenarzyści z największą ilością filmów

In [ ]:
#Scenarzyści z największą ilością filmów
imdb_movies['writer'].value_counts().head(10)

##TO DO  Reżyser - średni budżet filmów - SCENARZYŚCI I AKTORZY TEŻ

##Scenarzyści z największym dochodem na świecie - w dolarach

In [ ]:
#Scenarzysci z największym dochodem w dolarach
imdb_movies_USD.groupby('writer').agg({'imdb_title_id':'count','worlwide_gross_income':'sum'}).reset_index().rename(columns={'imdb_title_id':'films_count'}).sort_values(by='worlwide_gross_income', ascending=False).head(20)

##Scenarzyści największy dochód z filmów w USA

In [ ]:
#Scenarzyści z największym dochodem w dolarach
imdb_movies_USD.groupby('writer').agg({'imdb_title_id':'count','usa_gross_income':'sum'}).reset_index().rename(columns={'imdb_title_id':'films_count'}).sort_values(by='usa_gross_income', ascending=False).head(20)

#Aktorzy

In [ ]:
imdb_names.head()

# Aktorzy z największymi średnimi głosów, największa liczba głosów - EWA

In [ ]:
imdb_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [ ]:
imdb_movies_actors = imdb_movies.actors.str.split(pat=',', expand=True)
imdb_movies_actors['avg_vote'] = imdb_movies['avg_vote']
imdb_movies_actors

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,avg_vote
0,Blanche Bayliss,William Courtenay,Chauncey Depew,None,None,None,None,None,None,None,None,None,None,None,None,None,5.9
1,Elizabeth Tait,John Tait,Norman Campbell,Bella Cola,Will Coyne,Sam Crewes,Jack Ennis,John Forde,Vera Linden,Mr. Marshall,Mr. McKenzie,Frank Mills,Ollie Wilson,None,None,None,6.1
2,Asta Nielsen,Valdemar Psilander,Gunnar Helsengreen,Emil Albes,Hugo Flink,Mary Hagen,None,None,None,None,None,None,None,None,None,None,5.8
3,Helen Gardner,Pearl Sindelar,Miss Fielding,Miss Robson,Helene Costello,Charles Sindelar,Mr. Howard,James R. Waite,Mr. Osborne,Harry Knowles,Mr. Paul,Mr. Brady,Mr. Corker,None,None,None,5.2
4,Salvatore Papa,Arturo Pirovano,Giuseppe de Liguoro,Pier Delle Vigne,Augusto Milla,Attilio Motta,Emilise Beretta,None,None,None,None,None,None,None,None,None,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,Dany Boon,Philippe Katerine,Anne Serra,Samuel Jouy,Sophie Verbeeck,Carole Brana,Benoît Pétré,Aksel Ustun,Mathieu Lardot,Olivier Sa,Julien Prevost,Antoine Mathieu,David Ban,Stan,Guillaume Clémencin,None,5.3
85851,Herman Finkers,Johanna ter Steege,Leonie ter Braak,Stef Assen,Annie Beumers,Jos Brummelhuis,Reinier Bulder,Daphne Bunskoek,Karlijn Koel,Karlijn Lansink,Marieke Lustenhouwer,Jan Roerink,Ferdi Stofmeel,Aniek Stokkers,Belinda van der Stoep,None,7.7
85852,Anoop Chandran,Indrans,Sona Nair,Simon Britto Rodrigues,None,None,None,None,None,None,None,None,None,None,None,None,7.9
85853,Ahmet Faik Akinci,Belma Mamati,Metin Keçeci,Burhan Sirmabiyik,Orhan Aydin,Tevfik Yapici,Yusuf Eksi,Toygun Ates,Aziz Özuysal,Dilek Ölekli,Arcan Bunial,Seval Hislisoy,Ergül Çolakoglu,Gülçin Ugur,Ibrahim Balaban,None,6.4


In [ ]:
imdb_movies_actors[1][1] # ta spacja przed imieniem zniszczyła mi piątkowy wieczór

' John Tait'

In [ ]:
# tu usuwam spacje przed imionami we wszytskich kolumnach

def remove_white_space_before(x):
  if isinstance(x, str):
    if x[0] == ' ':
      x = x[1:]
  return x

for i in range(0,16):
  imdb_movies_actors[i] = imdb_movies_actors[i].apply(remove_white_space_before)

In [ ]:
imdb_movies_actors[1][1] # i działa <3

'John Tait'

In [ ]:
actors = {}
for i in range(0, 16):
  actors["act_column{0}".format(i)] = imdb_movies_actors[[i, 'avg_vote']].groupby(i).agg(['count', 'sum'])
  actors["act_column{0}".format(i)].columns = actors["act_column{0}".format(i)].columns.droplevel(0)
  actors["act_column{0}".format(i)].index.name = None

actors['act_column0']

,count,sum
'Lee' George Quinones,1,7.1
'University' Jeevan,1,5.7
'Weird Al' Yankovic,1,7.0
2Mex,1,3.8
50 Cent,5,23.7
...,...,...
Þorsteinn Bachmann,1,5.6
Þorsteinn Guðmundsson,1,6.1
Þröstur Leó Gunnarsson,2,11.9
Þórhallur Sigurðsson,1,5.7


In [ ]:
actors_all = actors['act_column0']
for i in range(1,16):
  actors_all = actors_all.add(actors["act_column{0}".format(i)], fill_value=0)

actors_all

,count,sum
'Astor' Escobar,1.0,6.4
'Baby' Carmen De Rue,3.0,15.5
'Bang Bang' Toto Ruta,1.0,7.1
'Big Al' Solomon,1.0,3.4
'Big Jack' Provan,1.0,6.0
...,...,...
Þórir Waagfjörð,1.0,5.3
Þórunn Antónía Magnúsdóttir,1.0,6.3
Þórunn Arna Kristjánsdóttir,1.0,7.1
Þórunn Magnea Magnúsdóttir,3.0,19.8


In [ ]:
actors_all['avg_vote'] = actors_all['sum']/actors_all['count']
actors_all

,count,sum,avg_vote
'Astor' Escobar,1.0,6.4,6.400000
'Baby' Carmen De Rue,3.0,15.5,5.166667
'Bang Bang' Toto Ruta,1.0,7.1,7.100000
'Big Al' Solomon,1.0,3.4,3.400000
'Big Jack' Provan,1.0,6.0,6.000000
...,...,...,...
Þórir Waagfjörð,1.0,5.3,5.300000
Þórunn Antónía Magnúsdóttir,1.0,6.3,6.300000
Þórunn Arna Kristjánsdóttir,1.0,7.1,7.100000
Þórunn Magnea Magnúsdóttir,3.0,19.8,6.600000


In [ ]:
# Aktorzy, którzy zagrali w więcej niż 15 filmach posortowani wg średniej oceny filmu

actors_all[actors_all['count'] > 15].sort_values(by='avg_vote', ascending=False).reset_index().head(10)

,index,count,sum,avg_vote
0,Rabi Ghosh,17.0,133.5,7.852941
1,Sergey A.,31.0,240.0,7.741935
2,Vyacheslav Tikhonov,16.0,123.0,7.687500
3,Toma Caragiu,17.0,129.5,7.617647
4,Charles Chaplin,18.0,137.1,7.616667
5,Sebastian Papaiani,26.0,197.2,7.584615
6,Yoshimasa Hosoya,17.0,128.7,7.570588
7,Bora Todorovic,19.0,143.8,7.568421
8,Sankaradi,45.0,339.5,7.544444
9,George Mihaita,16.0,120.7,7.543750


In [ ]:
# Aktorzy, którzy zagrali w więcej niż 50 filmach posortowani wg średniej oceny filmu

actors_all[actors_all['count'] > 50].sort_values(by='avg_vote', ascending=False).reset_index().head(10)

# Myślałam, że jakieś bardziej znane nazwiska się pojawią :D

,index,count,sum,avg_vote
0,Kamal Haasan,76.0,565.8,7.444737
1,Sener Sen,55.0,404.4,7.352727
2,Thilakan,69.0,506.5,7.340580
3,Kemal Sunal,78.0,568.0,7.282051
4,Pavle Vuisic,51.0,367.2,7.200000
5,Tatsuya Nakadai,58.0,415.3,7.160345
6,Bogdan Diklic,69.0,492.6,7.139130
7,Takashi Shimura,51.0,363.9,7.135294
8,Toshirô Mifune,62.0,437.7,7.059677
9,Sreenivasan,83.0,585.5,7.054217


In [ ]:
# Aktorzy, którzy zagrali w większej liczbie filmów

actors_all.sort_values(by='count', ascending=False).reset_index().head(10)

,index,count,sum,avg_vote
0,Brahmanandam,318.0,1892.5,5.951258
1,Anupam Kher,211.0,1195.3,5.664929
2,Nassar,204.0,1282.9,6.288725
3,Eric Roberts,204.0,904.0,4.431373
4,Prakash Raj,189.0,1169.9,6.189947
5,Tanikella Bharani,173.0,1076.7,6.223699
6,Amitabh Bachchan,173.0,1079.1,6.237572
7,Mohanlal,169.0,1120.3,6.628994
8,Gérard Depardieu,164.0,1000.1,6.098171
9,John Carradine,162.0,895.7,5.529012
